<a href="https://colab.research.google.com/github/mohitmethi1000/naive_bayes_classification/blob/main/spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

impoting the data from google drive

In [3]:
!pip install -q gdown
import gdown

file_id = "1Wj_H_C3z45UUPxRblLKSfZzZFWpnDyaS"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "spam_data.csv", quiet=False)

import pandas as pd
data = pd.read_csv("spam_data.csv")


Downloading...
From (original): https://drive.google.com/uc?id=1Wj_H_C3z45UUPxRblLKSfZzZFWpnDyaS
From (redirected): https://drive.google.com/uc?id=1Wj_H_C3z45UUPxRblLKSfZzZFWpnDyaS&confirm=t&uuid=4bead8bd-3614-416b-a449-94322cad62a5
To: /content/spam_data.csv
100%|██████████| 140M/140M [00:01<00:00, 84.6MB/s]


In [4]:
data.head()

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...


In [5]:
x = data['text']
y = data['label']

1 -> means spam

0 -> means ham

splitting the data

In [6]:
x_array = np.array(x)
y_array = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x_array, y_array, test_size=0.2, random_state=42)

converting the data in [0,0,0,1,0,0,0,1] - form

In [ ]:
cv = CountVectorizer(stop_words = 'english')
x_train = cv.fit_transform(x_train)
x_test = cv.transform(x_test)         # sparse matrix

In [ ]:
x_train.shape

(66758, 276601)

# implimenting multinomial naive bayes from sratch

In [ ]:
class MultinomialNB:
    def __init__(self):
        # initialize all model parameters
        self.classes = None
        self.class_count = {}
        self.word_count = {}
        self.feature_log_prob = {}
        self.class_log_prior = {}
        self.V = 0  # vocabulary size for laplace smoothing

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_count = {}
        self.word_count = {}
        self.feature_log_prob = {}
        self.class_log_prior = {}
        self.V = X.shape[1]

        for c in self.classes:
            X_c = X[y == c]
            self.class_count[c] = X_c.shape[0]
            self.word_count[c] = X_c.sum(axis=0)
            total_count = self.word_count[c].sum()
            # laplace smoothing
            self.feature_log_prob[c] = np.log((self.word_count[c] + 1) / (total_count + self.V))
            self.class_log_prior[c] = np.log(self.class_count[c] / len(y))

    def predict(self, X):
        predictions = []
        for i in range(X.shape[0]):
            row = X[i]
            log_probs = {}
            for c in self.classes:
                log_prob = self.class_log_prior[c]
                log_prob += row @ self.feature_log_prob[c].T
                log_probs[c] = log_prob
            predicted_class = max(log_probs, key=log_probs.get)
            predictions.append(predicted_class)
        return np.array(predictions)

    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

    def top_words(self, n = 5):
        # top words are with maximum p(x/y) for each class
        top_words_indices = {}
        # first geting top words
        for c in self.classes:
            top_words_indices[c] = np.argsort(self.feature_log_prob[c].A1)[-n:][::-1]

        # now getting top words for a particular class
        feature_names = cv.get_feature_names_out()
        top_words = {}
        for c in self.classes:
            top_words[c] = feature_names[top_words_indices[c]]
        return top_words


training the model

In [ ]:
model = MultinomialNB()
model.fit(x_train, y_train)
print('accuracy:', model.score(x_test, y_test))

accuracy: 0.9778310365488316


In [ ]:
mail = "your test is scheduled for tommorow"
vectorized = cv.transform([mail])
pred = model.predict(vectorized)
if pred == 1:
  print("spam")
else:
  print("ham")

ham


In [ ]:
top_words = model.top_words(n = 10)

In [ ]:
for c, words in top_words.items():
    print(f"Top words for class {c}: {', '.join(words)}")

Top words for class 0: escapenumber, http, enron, org, com, escapelong, ect, help, samba, list
Top words for class 1: escapenumber, escapelong, com, http, pills, escapenumbermg, price, company, save, item
